## Results and Evaluation

In the introduction it was mentioned that we are going to use Sharpe Ratio adjusted for skewness and kurtosis as a measure of loan application "goodness".

The Sharpe Ratio (Sharpe (1966)) is defined as:

$SR = \frac{r - i}{SD}$

where r stands for expected return (in our case mean of the distribution), i for risk free interest rate (we assumed it is 0), and SD for standard devition, as a measure of a risk.

However, as it was explained in the introduton, normal distribution of profit rates seems as unnecessary strong assumption. Hence, Sharpe Ratio, accounting just for mean and variance, is unsuitable.

In order to solve this we employ adjustment introduced by Pezier and White (2006), where in addition to mean and standard deviation, we take into account skewness and kurtosis of distribution. Additionally, it should be mentioned that the following formula assumes exponential utility function and further risk-seeking behaviour of economic agents. This assumption seem quite reasonable in our case. The Adjusted for Skewness and Kurtosis Sharpe Ratio (ASKSR):

$ASKSR = SR[1 + \frac{S}{6}SR - \frac{K - 3}{24} SR^{2}]$ 

Where K stands for kurtosis and S for skewness.

 Now let's see how our models perform. We calculated ASKSR for each applicant in our test dataset of 33000 observations. Then, on the first half of the test dataset, we found an optimal cutoff point for ASKSR. We did it by summing the profit that is above some assumed ASKSR level. Then, we evaluate the optimal cutoff on the second part of the test sample. We are expressing the result as a percantage of the maximum possible profit (i.e. sum of possitive profits).

In [1]:
import pickle
import numpy  as np
from scipy.stats import kurtosis, skew, mode
import pandas as pd
from sklearn.model_selection import train_test_split
import glob

In [2]:
def optimalCut (x,y):
        max_y = max(y)
        max_x = x[y.index(max_y)]
        return (max_x)

In [3]:
profit = np.array(pickle.load(open( "data/y_test_abs.pickle", "rb" )))

In [4]:
method = []
stacked_res = []
#MCDrop Results

#for filepath in glob.iglob('C:/master/iss/results/Batuhan/mcdrop//*.pickle'):
            
results = np.array(pickle.load(open( "results/bayesian/mc_dropout.pickle", "rb" )))

df = pd.DataFrame(
     {'mean': np.mean(results, axis=0)[:,0],
      'variance': np.var(results, axis=0)[:,0],
      'skewness': skew(results, axis = 0)[:,0],
      'kurtosis': kurtosis(results, axis = 0)[:,0],
      'Returns': profit
     })

df['standardDeviation'] = np.sqrt(df['variance'])
df['SR_mean'] = df['mean']/df['standardDeviation']
df['ASKSR_mean'] = df['SR_mean'] * (1 + df['skewness'] * df['SR_mean']/6 - (df['kurtosis'] - 3)/24 * (df['SR_mean'] ** 2))

    
df, df_test = train_test_split(df, test_size=0.5, random_state=42)

ASKSR_cutoff_mean = np.linspace(min(df['ASKSR_mean']), max(df['ASKSR_mean']), 10000)

ASKSR_profit_mean = []

for i in range(0, len(ASKSR_cutoff_mean) - 1, 1):
    ASKSR_profit_mean.append(df.loc[df['ASKSR_mean'] > ASKSR_cutoff_mean[i], 'Returns'].sum())   
        
    
ASKSR_mean_Optimal = optimalCut(x = ASKSR_cutoff_mean, y = ASKSR_profit_mean)

ASKSR_profit_optimal_mean = []
ASKSR_profit_optimal_mean.append(df_test.loc[df_test['ASKSR_mean'] > ASKSR_mean_Optimal, 'Returns'].sum())    
    
method.append("MC Droput")
stacked_res.append(ASKSR_profit_optimal_mean)

In [5]:
#VI Results

#for filepath in glob.iglob('C:/master/iss/results/Batuhan/mcdrop//*.pickle'):
            
    results = np.array(pickle.load(open( "results/bayesian/VariationalInference.pickle", "rb" )))

    df = pd.DataFrame(
        {'mean': np.mean(results, axis=0),
         'variance': np.var(results, axis=0),
         'skewness': skew(results, axis = 0),
         'kurtosis': kurtosis(results, axis = 0),
         'Returns': profit
        })

    df['standardDeviation'] = np.sqrt(df['variance'])
    df['SR_mean'] = df['mean']/df['standardDeviation']
    df['ASKSR_mean'] = df['SR_mean'] * (1 + df['skewness'] * df['SR_mean']/6 - (df['kurtosis'] - 3)/24 * (df['SR_mean'] ** 2))

    
    df, df_test = train_test_split(df, test_size=0.5, random_state=42)

    ASKSR_cutoff_mean = np.linspace(min(df['ASKSR_mean']), max(df['ASKSR_mean']), 10000)

    ASKSR_profit_mean = []

    for i in range(0, len(ASKSR_cutoff_mean) - 1, 1):
        ASKSR_profit_mean.append(df.loc[df['ASKSR_mean'] > ASKSR_cutoff_mean[i], 'Returns'].sum())   
        
    
    ASKSR_mean_Optimal = optimalCut(x = ASKSR_cutoff_mean, y = ASKSR_profit_mean)

    ASKSR_profit_optimal_mean = []
    ASKSR_profit_optimal_mean.append(df_test.loc[df_test['ASKSR_mean'] > ASKSR_mean_Optimal, 'Returns'].sum())    
    
    method.append("Variational Inference")
    stacked_res.append(ASKSR_profit_optimal_mean)

In [10]:
results = np.array(pickle.load(open( "results/deterministic/Regularized.pickle", "rb" )))

df = pd.DataFrame(
        {'mean': results[:,0],
         'Returns': profit
        })

df, df_test = train_test_split(df, test_size=0.5, random_state=42)

deterministic_cutoff_mean = np.linspace(min(df['mean']), max(df['mean']), 10000)

deterministic_profit_mean = []

for i in range(0, len(deterministic_cutoff_mean) - 1, 1):
    deterministic_profit_mean.append(df.loc[df['mean'] > deterministic_cutoff_mean[i], 'Returns'].sum())   

deterministic_mean_Optimal = optimalCut(x = deterministic_cutoff_mean, y = deterministic_profit_mean)

deterministic_profit_optimal_mean = []
deterministic_profit_optimal_mean.append(df_test.loc[df_test['mean'] > deterministic_mean_Optimal, 'Returns'].sum())    
    
method.append("Deterministic")
stacked_res.append(deterministic_profit_optimal_mean)    
    

In [11]:
method.append("Naive")
stacked_res.append(sum(df_test['Returns']))

method.append("Max")
stacked_res.append(df_test.loc[df_test['Returns'] > 0, 'Returns'].sum())

In [12]:
results_Compare = pd.DataFrame(
        {'method': method,
         'result': stacked_res
        })

In [14]:
for j in range (0,3):
    results_Compare['result'][j] = results_Compare['result'][j][0] 

In [15]:
for m in range (0,4):
    results_Compare['result'][m] = results_Compare['result'][m]/results_Compare['result'][4]   

The results are presented in the next table:

In [16]:
results_Compare

,method,result
0,MC Droput,0.234959
1,Variational Inference,0.224832
2,Deterministic,0.231184
3,Naive,0.037151
4,Max,2.55511e+07


As we can see, MC Dropout beats both Deterministic approach and VI by a margin, while all three are way better than naive approach.

## Conclusion

In this study, we tried to show that uncertainty is not only something that places obstacle in front of good predictions. We can indeed benefit from it, if we are able to estimate it. It is a difficult task though. Approximate Bayesian variational inference techniques provide us with scalable estimations. Among them, MC dropout yields results those are very close to usual neural network models in performance. MC dropout is easier to use, faster and more scalable. We apply it very intuitively apply it on both training and test times and automatically obtain an approximate probabilistic model.But is just one way of approximating the posterior. Applications of Bayesian inference in deep learning is a reviving field, and now in development. In near future, we are likely to witness better theoretical explorations and algorithmic developments. Not to say further successful applications in finance and many other fields

## Bibliography

* Knight, F. (1921). From Risk, Uncertainty, and Profit. The Economic Nature of the Firm.

* Gal., Y.  (2015, July 3). What My Deep Model Doesn’t Know [Web log post]. Retrieved February 8, 2017, from        http://mlg.eng.cam.ac.uk/yarin/blog_3d801aa532c1ce.html

* Bernardo, J. M., & Smith, A. F. (1994). Bayesian Theory. Wiley Series in Probability and Statistics.

* Neal, R. M. (1996). Bayesian Learning for Neural Networks. Lecture Notes in Statistics.

* Betancourt, M., & Girolami, M. (2015). Hamiltonian Monte Carlo for Hierarchical Models. Current Trends in Bayesian  Methodology with Applications. doi:10.1201/b18502-5

* Robert, C. P., & Casella, G. (2004). Monte Carlo Optimization. Springer Texts in Statistics Monte Carlo Statistical Methods.

* Murphy, K. P. (2012). Machine learning: A probabilistic perspective. Cambridge, England: The MIT Press.

* Graves, A. (2011). Practical variational inference for neural networks. Advances in Neural Information Processing Systems.

* Ranganath, R., Gerrish, S., & Blei, D. M. (2014). Black Box Variational Inference. Proceedings of the 17th International Conference on Artificial Intelligence and Statistics (AISTATS), 33.

* Blei, D. M., Kucukelbir, A., & Mcauliffe, J. D. (2017). Variational Inference: A Review for Statisticians. Journal of the American Statistical Association, 112(518), 859-877.

* Gal, Y., & Ghahramani, Z. (2016). Dropout as a Bayesian Approximation: Representing Model Uncertainty in Deep Learning. Proceedings of the 33 Rd International Conference on Machine Learning, 43.

* Srivastava, N., Hinton, G., Krizhevsky, A., Sutskever, I., &  Salakhutdinov, R. (2014). Dropout: a simple way to prevent neural networks from overfitting. Journal of Machine Learning Research, 15 (1).

* MacKay, D. J. (2004). Information theory, inference, and learning algorithms. Cambridge, U.K.: Cambridge University Press.

* Osband, I. (2016). Risk versus Uncertainty in Deep Learning : Bayes , Bootstrap and the Dangers of Dropout.

* Sharpe, W. F. (1966). "Mutual Fund Performance". Journal of Business. 39 (S1): 119–138. doi:10.1086/294846.

* Pezier J, & White A, (2006) "The relative Merits of Investable Hedge Fund indices and of Funds of Hedge Funds in Optimal Passive Portfolios"

* Andreeva, G., Ansell, J., & Crook, J. (2007). Modelling profitability using survival combination scores. European Journal of Operational Research, 183, 1537-1549.

* R T Stewart (2011) A profit-based scoring system in consumer credit: making acquisition decisions for credit cards, Journal of the Operational Research Society, 62:9, 1719-1725, DOI: 10.1057/jors.2010.135 

* S M Finlay (2008) Towards profitability: a utility approach to the credit scoring problem, Journal of the Operational Research Society, 59:7, 921-931, DOI: 10.1057/ palgrave.jors.2602394

* Luis Javier Sánchez Barrios, Galina Andreeva & Jake Ansell (2014) Monetary and relative scorecards to assess profits in consumer revolving credit, Journal of the Operational Research Society, 65:3, 443-453, DOI: 10.1057/jors.2013.66 

* Carlos Serrano-Cinca-Begoña Gutiérrez-Nieto (2016),  The use of profit scoring as an alternative to credit scoring systems in peer-to-peer (P2P) lending, Decision Support Systems, https://doi.org/10.1016/j.dss.2016.06.014

* Thomas Verbraken, Cristian Bravo, Richard Weber, Bart Baesens (2014), Development and application of consumer credit scoring models using profit-based classification measures, European Journal of Operational Research

* Stefan Lessmann, Bart Baesens, Hsin-Vonn Seow, Lyn C. Thomas (2015), Benchmarking state-of-the-art classification algorithms for credit scoring: An update of research, European Journal of Operational Research

* Mee Chi So, Lyn C. Thomas, Hsin-Vonn Seow, Christophe Mues (2014), Using a transactor/revolver scorecard to make credit and pricing decisions, Decision Support Systems

* Kaveh Bastani, Elham Asgari, Hamed Namavari (2018), Wide and Deep Learning for Peer-to-Peer Lending, arXiv

* Selçuk Bayracı (2017), Application of profit-based credit scoring models using R, Romanian Statistical Review 4/2017

* Devavrat Shah and Kang Zhang (2014), Bayesian regression and Bitcoin, 52nd Annual Allerton Conference on Communication, Control, and Computing (Allerton)

* Catarina Moreira and Emmanuel Haven and Sandro Sozzo and Andreas Wichert (2017), The Dutch's Real World Financial Institute: Introducing Quantum-Like Bayesian Networks as an Alternative Model to deal with Uncertainty, arXiv

* Michael Maio Pires and Tshilidzi Marwala (2007), Option Pricing Using Bayesian Neural Networks, arXiv